In [4]:
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json

In [24]:
import glob
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

In [57]:
def read_text_file(file_path,index,image_name,image_shape):# YOLO Label을 읽기 위한 함수
    dataDir = './../../../COCO_dataset'
    dataType = 'val2017'
    annFile = '{}/annotation/instances_{}.json'.format(dataDir,dataType)
    print(annFile)
    width , height = image_shape
    label = []
    with open(annFile) as json_file:
        json_data = json.load(json_file)
        json_image = json_data["images"]
        for i in json_image:
            if(i["file_name"]==image_name):
                id = i["id"]
        json_bbox = json_data["annotations"]
        for i in json_bbox:
            if(i["image_id"]==id):
                bbox = i["bbox"]
        print(bbox) # x ,y , width , height
        for i in range(0,len(bbox),4):
            label_x = (bbox[i] + bbox[i+2]/2)/width
            label_y = (bbox[i+1] + bbox[i+3]/2)/height
            label_w = bbox[i+2]/width
            label_h = bbox[i+3]/height
            label.append({"classes": 0, "x": label_x, "y":label_y,"width":label_w,"height":label_h.strip('\n')})
    return label

In [58]:
class ImageDataset_withLabel(Dataset):
    def __init__(self, root, resize_shape, root_label):
        self.root = root+'\\'
        resize_height, resize_width = resize_shape
        self.resize_transform = transforms.Compose(
            [
                transforms.Resize((resize_height,resize_width), Image.NEAREST), # image resize
                transforms.ToTensor(), # float to tensor
                transforms.Normalize(mean,std), # Normalize
            ]
        )

        self.image_files = sorted(glob.glob(root+"/*.jpg")) # image sort
        self.label_files = sorted(glob.glob(root_label+"/*.txt")) # label sort

    def __getitem__(self,index):
        img = Image.open(self.image_files[index % len(self.image_files)])
        print(self.root)
        img_name = self.image_files[index%len(self.image_files)].replace(self.root,"")
        print("img : ",img_name)
        origin_width , origin_height = img.size
        origin_img = np.array(img.getdata()).reshape(img.size[0], img.size[1],-1)

        img_data = self.resize_transform(img)
        label = read_text_file(self.label_files,index % len(self.image_files),img_name,[origin_width,origin_height])

        return {"img": img_data, "label": label, "origin_width":origin_width,"origin_height":origin_height,"origin_img":origin_img}

    def __len__(self):  # data size를 넘겨주는 파트
        return len(self.image_files)  # 파일 길이 반환 ( 총 이미지 수 )


In [59]:
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from utils.datasets.readDatasets import *
import cv2

#print(read_text_file("./../../test.txt"))

if __name__ == '__main__':
    dataloader = DataLoader(  # training data read
            ImageDataset_withLabel(root="../../../COCO_dataset/val2017" , resize_shape=(416,416),root_label="../../../COCO_dataset/annotation"),# root = ../../data/img_align_celeba &  hr_shape = hr_shape
            batch_size=1,  # batch size ( mini-batch )
            shuffle=True,  # shuffle
            num_workers=1, # using 8 cpu threads
        )

    for i, dataset in enumerate(dataloader):
        print("i:{} ==> img.shape :{},\nlabel :{}".format(i,dataset["img"],dataset["label"]))
        print(len(dataset["label"]))
        #기존 이미지 불러오기
        origin_img = dataset["origin_img"]
        print("origin_img's shape : ",origin_img.shape)
        origin_img = np.squeeze(origin_img, axis=0) # 차원 삭제 axis = 0 ==> 가장 맨 앞의 배열 차원을 없앤다.
        print("origin_img's shape : ", origin_img.shape)
        origin_img = np.array(origin_img, dtype='uint8') # pil저장 시 자료형태는 uint8 or float이여야 한다.
        origin_img = np.reshape(origin_img,[dataset["origin_height"],dataset["origin_width"],-1])
        #plt.imshow(origin_img)
        #plt.show()
        for j in dataset["label"]:
            print(j) # 전체 map 정보 출력
            print('classes : {}'.format(j["classes"])) # 각 key를 통한 value 접근

            #현재 크기에 맞게 label값 변경
            x = float(j["x"][0]) * dataset["origin_width"]
            y = float(j["y"][0]) * dataset["origin_height"]
            width = float(j["width"][0]) * dataset["origin_width"]
            height = float(j["height"][0]) * dataset["origin_height"]

            print("x : {}, y : {}, width : {}, height : {}".format(x,y,width,height))
            #rectangle을 그리기 위한 x1,y1 & x2,y2 값
            x1 = x - width/2
            y1 = y - height/2
            x2 = x + width/2
            y2 = y + height/2

            cv2.rectangle(origin_img, (x1,y1), (x2,y2), (0, 0, 255), 1) # label을 통하여 boundingBox그리기
            # cv.rectangle(img,(x1,y1),(x2,y2),color,thickness,lineType,shift)
        save_image(dataset["img"],"./%d.jpg"%i)
        #cv shape = (height, width, channel)
        plt.imshow(origin_img)
        plt.show()
        plt.imsave('./newImage.jpg',origin_img)

ModuleNotFoundError: No module named 'utils'

In [2]:
dataDir = './../../../COCO_dataset'
dataType = 'val2017'
annFile = '{}/annotation/instances_{}.json'.format(dataDir,dataType)

In [3]:
print(annFile)

./../../../COCO_dataset/annotation/instances_val2017.json


In [9]:
with open(annFile) as json_file:
    json_data = json.load(json_file)

In [47]:
    json_image = json_data["images"]
    for i in json_image:
        if(i["file_name"]=="000000021604.jpg"):
            id = i["id"]

In [48]:
    print(id)

21604


In [49]:
    json_bbox = json_data["annotations"]
    for i in json_bbox:
        if(i["image_id"]==id):
            bbox = i["bbox"]

In [50]:
    print(bbox) # x ,y , width , height

[41.71, 47.46, 438.65, 581.03]
